In [3]:
! ls -al /root/PySpark/workspace/7_Airflow/SparkSubmitOperator/

total 56
drwxr-xr-x 3 root root  4096 Mar 25 21:16 .
drwxr-xr-x 7 root root  4096 Mar 25 21:18 ..
-rw-r--r-- 1 root root 11528 Mar 25 16:58 basicsparksubmit.ipynb
-rw-r--r-- 1 root root   499 Mar 20 06:36 basicsparksubmit.py
-rw-r--r-- 1 root root  1188 Mar 20 04:26 dag_basicsparksubmit.py
drwxr-xr-x 2 root root  4096 Mar 25 21:16 .ipynb_checkpoints
-rw-r--r-- 1 root root  1779 Mar 15 01:23 spark_dag.py
-rw-r--r-- 1 root root  2445 Mar 20 07:34 step1_preprocess.py
-rw-r--r-- 1 root root  9677 Mar 20 08:12 step1_preprocess_sparksubmit.ipynb
-rw-r--r-- 1 root root    72 Mar 25 21:16 Untitled.ipynb


In [44]:
%%writefile pi.py
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

import sys
from random import random
from operator import add

from pyspark.sql import SparkSession

if __name__ == "__main__":
    """
        Usage: pi [partitions]
    """

    spark = SparkSession\
        .builder\
        .appName("PythonPi")\
        .getOrCreate()

    partitions = int(sys.argv[1]) if len(sys.argv) > 1 else 2
    n = 100000 * partitions

    def f(_: int) -> float:
        x = random() * 2 - 1
        y = random() * 2 - 1
        return 1 if x ** 2 + y ** 2 <= 1 else 0

    count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
    print("Pi is roughly %f" % (4.0 * count / n))

    spark.stop()
    


Overwriting pi.py


In [40]:
#! docker exec master /opt/hadoop/bin/hdfs dfsadmin -safemode leave
#! docker exec master /opt/hadoop/bin/hdfs dfsadmin -safemode forceExit

In [45]:
! docker exec master /opt/hadoop/bin/hdfs dfs -ls /

Found 5 items
drwxr-xr-x   - root supergroup          0 2022-03-18 01:48 /apps
-rw-r--r--   3 root supergroup    2397103 2022-03-26 09:23 /database.csv
drwxr-xr-x   - root supergroup          0 2022-03-18 01:49 /spark-jars
drwxr-xr-x   - root supergroup          0 2022-03-26 09:33 /spark-logs
drwx-wx-wx   - root supergroup          0 2022-03-18 01:41 /tmp


In [46]:
! docker exec master /opt/hadoop/bin/hdfs dfs -mkdir -p /SparkSubmitOperator/

In [47]:
! docker cp pi.py master:/root/
! docker exec master /opt/hadoop/bin/hdfs dfs -rm /SparkSubmitOperator/pi.py
! docker exec master /opt/hadoop/bin/hdfs dfs -put /root/pi.py /SparkSubmitOperator/pi.py
! docker exec master /opt/hadoop/bin/hdfs dfs -cat /SparkSubmitOperator/pi.py

rm: `/SparkSubmitOperator/pi.py': No such file or directory
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

import sys
from random import random
from operator import add

from pyspark.sql import SparkSession

if __name__ == "__main__":
    """
        Usage: pi [

In [48]:
%%writefile ../../dags/spark_dag.py
#
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.

"""
Example Airflow DAG to submit Apache Spark applications using
`SparkSubmitOperator`, `SparkJDBCOperator` and `SparkSqlOperator`.
"""
from airflow.models import DAG
from airflow.providers.apache.spark.operators.spark_submit import SparkSubmitOperator
from airflow.utils.dates import days_ago

args = {
    'owner': 'Airflow',
}

with DAG(
    dag_id='spark_submit_operator',
    default_args=args,
    schedule_interval=None,
    start_date=days_ago(2),
    tags=['example'],
) as dag:    
    # [START howto_operator_spark_submit]
    
    python_submit_job = SparkSubmitOperator(
        application="hdfs://master:9000/SparkSubmitOperator/pi.py", task_id="python_job"
    )
    
    # scala_submit_job = SparkSubmitOperator(
    #     application="/workspace/example-airflow-and-spark/airflow-spark-assembly-0.1.0-SNAPSHOT.jar", task_id="scala_job"
    # )

    # python_submit_job >> scala_submit_job
    
    # [END howto_operator_spark_submit]


Overwriting ../../dags/spark_dag.py
